# Setup

In [133]:
# !pip install "modin[all]"
# !pip install pandas==1.4.2
# !pip install ipyfilechooser
# !pip install duckdb==0.3.4
# !pip install pyarrow
# !pip install swifter
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Import

In [172]:
import modin.pandas as pd
import datetime
import itertools
from ipyfilechooser import FileChooser
import duckdb
import pyarrow as pa
from pyarrow import csv
import pyarrow.dataset as ds
import swifter

# Widgets

## DB

## CSV

In [15]:
# Create and display a FileChooser widget
csv_chooser = FileChooser('.')
csv_chooser.title = '<b>Upload CSV</b>'
csv_chooser.filter_pattern = "*.csv"
display(csv_chooser)

FileChooser(path='/Users/jujutan/projects/personal/restaurant_status', filename='', title='<b>Upload CSV</b>',…

In [18]:
# Create and display a FileChooser widget
db_chooser = FileChooser('.')
db_chooser.title = '<b>Upload DB (optional)</b>'
db_chooser.filter_pattern = "*.csv"
display(db_chooser)

FileChooser(path='/Users/jujutan/projects/personal/restaurant_status', filename='', title='<b>Upload DB (optio…

# Input Resolution

In [34]:
input_file_path = csv_chooser.selected
db_file_path = db_chooser.selected or 'duck.db'
db_conn = duckdb.connect(db_file_path)
read_options = csv.ReadOptions(
               column_names=["dining_place_name", "opening_time"],
               skip_rows=1)
input_csv = csv.read_csv(input_file_path, read_options)
df = input_csv.to_pandas()

In [38]:
df.head()

,dining_place_name,opening_time
0,Osakaya Restaurant,"Mon-Thu, Sun 11:30 am - 9 pm / Fri-Sat 11:30 ..."
1,The Stinking Rose,"Mon-Thu, Sun 11:30 am - 10 pm / Fri-Sat 11:30..."
2,McCormick & Kuleto's,"Mon-Thu, Sun 11:30 am - 10 pm / Fri-Sat 11:30..."
3,Mifune Restaurant,Mon-Sun 11 am - 10 pm
4,The Cheesecake Factory,Mon-Thu 11 am - 11 pm / Fri-Sat 11 am - 12:30...


# Functions

In [229]:
dow_map = ['mon', 'tue', 'wed', 'thu', 'fri', 'sat', 'sun']

def parse_timing(timing):
    if (not timing):
        return ""
    time = timing.strip().split('-')
    parsed_timing = []
    for t in time:
        parsed_time = t.strip().split(' ')
        unit_time = parsed_time[0]
        hour = unit_time.split(":")[0]
        minute = "00"
        if (":" in unit_time):
            minute = unit_time.split(":")[-1]
        hasPM = len(parsed_time) > 1 and ("pm" in parsed_time[1] or "PM" in parsed_time[1])
        if (hasPM):
            hour = str(int(hour) + 12)
        operating_timing = hour.zfill(2) + minute if hour else ''
        parsed_timing.append(operating_timing)
    return '-'.join(parsed_timing)
            

def process_section(section, opening_time):
    operating_time = section[-1].strip().split(" ", 1)[-1]
    # print("Time", operating_time)
    for i in section:
        days = i.strip().split(" ")[0].split("-")
        # print(days)
        if (len(days) > 1) and operating_time:
            start_idx = dow_map.index(days[0].lower())
            end_idx = dow_map.index(days[-1].lower()) + 1
            opening_time[start_idx:end_idx] = [operating_time] * (end_idx - start_idx)
        elif operating_time:
            opening_time[dow_map.index(days[0].lower())] = operating_time
    return opening_time
    
    
def parse_time(row):
    # print("Row", row)
    operating_time = [''] * 7
    blocks = row.strip().split("/")
    sections = list(itertools.chain(*[x.strip().split(",") for x in blocks]))
    sections = [process_section(x.strip().split(","), operating_time) for x in blocks]
    return sections[-1]

In [230]:
df['parsed_timing'] = df.apply(lambda x: ','.join([parse_timing(i) for i in parse_time(x['opening_time'])]), axis=1)

In [124]:
df

,dining_place_name,opening_time,parsed_timing
0,Osakaya Restaurant,"Mon-Thu, Sun 11:30 am - 9 pm / Fri-Sat 11:30 ...","1130-2100,1130-2100,1130-2100,1130-2100,1130-2..."
1,The Stinking Rose,"Mon-Thu, Sun 11:30 am - 10 pm / Fri-Sat 11:30...","1130-2200,1130-2200,1130-2200,1130-2200,1130-2..."
2,McCormick & Kuleto's,"Mon-Thu, Sun 11:30 am - 10 pm / Fri-Sat 11:30...","1130-2200,1130-2200,1130-2200,1130-2200,1130-2..."
3,Mifune Restaurant,Mon-Sun 11 am - 10 pm,"1100-2200,1100-2200,1100-2200,1100-2200,1100-2..."
4,The Cheesecake Factory,Mon-Thu 11 am - 11 pm / Fri-Sat 11 am - 12:30...,"1100-2300,1100-2300,1100-2300,1100-2300,1100-1..."
5,New Delhi Indian Restaurant,Mon-Sat 11:30 am - 10 pm / Sun 5:30 pm - 10 pm,"1130-2200,1130-2200,1130-2200,1130-2200,1130-2..."
6,Iroha Restaurant,"Mon-Thu, Sun 11:30 am - 9:30 pm / Fri-Sat 11:...","1130-2130,1130-2130,1130-2130,1130-2130,1130-2..."
7,Rose Pistola,Mon-Thu 11:30 am - 10 pm / Fri-Sun 11:30 am -...,"1130-2200,1130-2200,1130-2200,1130-2200,1130-2..."
8,Alioto's Restaurant,Mon-Sun 11 am - 11 pm,"1100-2300,1100-2300,1100-2300,1100-2300,1100-2..."
9,Canton Seafood & Dim Sum Restaurant,Mon-Fri 10:30 am - 9:30 pm / Sat-Sun 10 am - ...,"1030-2130,1030-2130,1030-2130,1030-2130,1030-2..."


In [170]:
import ipywidgets as widgets
import ipydatetime


day_selector = widgets.SelectMultiple(
    options=['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'],
    description='Day of Week',
    disabled=False
)
start_time_picker = ipydatetime.TimePicker()
end_time_picker = ipydatetime.TimePicker()
is_open_now = widgets.Checkbox(
    value=False,
    description="Show me what's open now",
    disabled=False,
    indent=False
)

is_open_now
day_selector
start_time_picker
end_time_picker

Checkbox(value=False, description="Show me what's open now", indent=False)

SelectMultiple(description='Day of Week', options=('Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'), value=())

TimePicker(value=None, step=60.0)

TimePicker(value=None, step=60.0)

In [159]:
filtered_day = [dow_map.index(x.lower()) for x in list(day_selector.value)]
filtered_start_time = start_time_picker.value.strftime("%H%M") if start_time_picker.value else ""
filtered_end_time = end_time_picker.value.strftime("%H%M") if end_time_picker.value else ""

In [185]:
filtered_day
filtered_start_time
filtered_end_time
is_open_now.value

[2]

'1230'

''

True

In [246]:
def filter_row(row, is_open_now, filtered_day, start_time, end_time):
    today = datetime.datetime.today()
    current_dow = today.weekday()
    current_time = today.strftime('%H%M')
    timing = row['parsed_timing']
    matched_day = True
    matched_time = True
    if (is_open_now):
        filtered_day = [current_dow]
        start_time = current_time
    if len(filtered_day) > 0:
        open_dining = [timing.split(',')[x] for x in filtered_day]
        matched_day = '' not in open_dining
        print('Matched Day', matched_day, filtered_day)
        if (matched_day):
            start_can = True
            end_can = True
            for i in filtered_day:
                opening_time = timing.split(',')[i]
                start = opening_time.split('-')[0]
                end = opening_time.split('-')[1] if len(opening_time.split('-')) > 1 else ''
                print(parse_time(row['opening_time']), row['opening_time'], timing, opening_time, start, start_time)
                if (start_time != '' and start != '' and start_time < start):
                    start_can =  False
                if (end_time != '' and end != '' and end_time > end):
                    end_can = False
            matched_time = start_can and end_can
        else:
            return False
    print(matched_day, matched_time)
    return matched_day and matched_time
        
    

In [247]:
filtered_rows = df.apply(lambda x: filter_row(x, is_open_now.value, filtered_day, filtered_start_time, filtered_end_time), axis=1)
df[filtered_rows]

Matched Day True [0]
['11:30 am - 9 pm', '11:30 am - 9 pm', '11:30 am - 9 pm', '11:30 am - 9 pm', '11:30 am - 9:30 pm', '11:30 am - 9:30 pm', '11:30 am - 9 pm'] Mon-Thu, Sun 11:30 am - 9 pm  / Fri-Sat 11:30 am - 9:30 pm 1130-2100,1130-2100,1130-2100,1130-2100,1130-2130,1130-2130,1130-2100 1130-2100 1130 0000
True False
Matched Day True [0]
['11:30 am - 10 pm', '11:30 am - 10 pm', '11:30 am - 10 pm', '11:30 am - 10 pm', '11:30 am - 11 pm', '11:30 am - 11 pm', '11:30 am - 10 pm'] Mon-Thu, Sun 11:30 am - 10 pm  / Fri-Sat 11:30 am - 11 pm 1130-2200,1130-2200,1130-2200,1130-2200,1130-2300,1130-2300,1130-2200 1130-2200 1130 0000
True False
Matched Day True [0]
['11:30 am - 10 pm', '11:30 am - 10 pm', '11:30 am - 10 pm', '11:30 am - 10 pm', '11:30 am - 11 pm', '11:30 am - 11 pm', '11:30 am - 10 pm'] Mon-Thu, Sun 11:30 am - 10 pm  / Fri-Sat 11:30 am - 11 pm 1130-2200,1130-2200,1130-2200,1130-2200,1130-2300,1130-2300,1130-2200 1130-2200 1130 0000
True False
Matched Day True [0]
['11 am - 10 pm'

,dining_place_name,opening_time,parsed_timing
